In [ ]:
# In terminal:
# pip install bqplot==0.12.2
# jupyter labextension list
# jupyter labextension install @jupyter-widgets/jupyterlab-manager 
# jupyter labextension install bqplot

In [ ]:
import pandas as pd
import numpy as np
import bqplot
import traitlets
import ipywidgets

# Dataset

[UFO dataset](https://github.com/planetsig/ufo-reports)

In [ ]:
# !wget https://github.com/planetsig/ufo-reports/raw/master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv

In [ ]:
ufo = pd.read_csv('ufo-scrubbed-geocoded-time-standardized.csv', 
                  names=['date_sighted', 'city', 'state', 'country',
                         'shape', 'duration', 
                         'duration_txt', 'note', 'date_reported', 
                         'latitude', 'longitude'],
                  parse_dates=['date_sighted', 'date_reported'])

ufo = ufo.reset_index().rename(columns={'index':'ufo_id'})
print(ufo.info())

In [ ]:
def find_dirty_data(col):
    for i, val in enumerate(col):
        try:
            float(val)
        except:
            print('Row {} has dirty data: {}'.format(i, [val]))

In [ ]:
find_dirty_data(ufo['duration'])

In [ ]:
find_dirty_data(ufo['latitude'])

In [ ]:
ufo = ufo.loc[~ufo.index.isin([27822, 35692, 58591, 43782])]
df = ufo.sample(n=1000, random_state=5)

In [ ]:
df['date_sighted'] = df['date_sighted'].str.replace('24:00', '00:00')
df['date_sighted'] = pd.to_datetime(df['date_sighted'])

df['duration'] = df['duration'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df.info()

# Basic plot types

# Lines: Number of UFOs sighted in each year

In [ ]:
# Prep data


In [ ]:
# A line plot 


In [ ]:
# See what can be controlled in Marks


In [ ]:
# See what can be controlled in y Axis


In [ ]:
# A line plot 
# - Add points to line
# - Change color
# - rotate x ticks
# - change x ticks text-anchor
# - x label offset


In [ ]:
# We can also do these things inline


# Bars: Shapes of UFOs

In [ ]:
# Prep Data

In [ ]:
# A Bar Chart


# Historgram: Duration in seconds

In [ ]:
# A Hist


# Scatter: Year sighted and Year reported

In [ ]:
# A basic one


In [ ]:
# A scatter plot colored by duration in seconds


# Interaction

# Pan-zoom

In [ ]:
# Add pan-zoom to line plot


# Tooltip

In [ ]:
# See what can be controlled in lines

In [ ]:
# Add tooptip to line plot


# Regenerating hist: Take user input as the number of bins

In [ ]:
# Interactively changing number of bins

# See what can be controlled in hist


In [ ]:
# Change number of bins


In [ ]:
# ipywidgets.Textarea
# ipywidgets.Button


In [ ]:
# Change number of bins by user input



# Brush selection: Selet an area in a scatter plot

In [ ]:
# Brush, along x



In [ ]:
# Brush, along y



In [ ]:
# Brush selection, 2D


# Filtering: Filter by duration in seconds

In [ ]:
# A slider

In [ ]:
# See the slider value


In [ ]:
# Scatter with slider
# A Scatter with scaled colors
